In [15]:
import pandas as pd
import ast

# Set file name
filename = 'US8695121B2_A42B3'
#filename = 'US9468782B2_A62B23'
#filename = 'US11133720B2_H02K3'
#filename = 'US11673469B2_B60K37'
#filename = 'US20200074811A1_G07F17'

df = pd.read_excel(f'/home/fantoni/patent-sentence-classification/results/patents/{filename}.xlsx')
df["probabilities"] = df["probabilities"].apply(ast.literal_eval) # import as a list

def calculate_score(probabilities):
    FUN, STR, MIX, OTH = probabilities
    # Check if OTH has the highest probability
    if OTH == max(probabilities):
        return -1  # set and invalid score
    
    score = FUN - STR + 0.5 * MIX
    normalized_score = (score + 1) / 2
    return round(normalized_score, 3)
    
df['score'] = df['probabilities'].apply(calculate_score)
df['probabilities'] = df['probabilities'].apply(lambda x: "; ".join(map(str, x))) # convert probabilities back to string, otherwise it gets error later!

In [16]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from matplotlib.colors import rgb2hex, LinearSegmentedColormap

# Create an Excel workbook and worksheet
wb = Workbook()
ws = wb.active
ws.title = "Title"
ws.append(df.columns.tolist())

def get_color(value):    
    # Define a colormap from blue (0) to red (1)
    cmap = LinearSegmentedColormap.from_list("blue_to_red", ["#0000FF", "#FF0000"])
    rgb = cmap(value)  # Get color for normalized score
    hex_color = rgb2hex(rgb)  # Convert to hex
    return PatternFill(start_color=hex_color[1:], end_color=hex_color[1:], fill_type="solid")

# Write data and apply colors to the "score" column
for row_idx, row in df.iterrows():
    row_data = row.tolist()
    ws.append(row_data)
    if row["score"] != -1:  #
        # Apply color to the "sent" column (index 2 in DataFrame)
        score_cell = ws.cell(row=row_idx + 2, column=3)  # Adjust for header (row starts at 2, the offset for header)
        score_cell.fill = get_color(row["score"])

# Save the Excel file
file_path = f'/home/fantoni/patent-sentence-classification/results/patents/{filename}_color.xlsx'
wb.save(file_path)